In [7]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

In [11]:
df = pd.read_csv('./csv/preprocessed_data.csv')
df.head()

,Group,Rt_scale_IKI-mean,Rt_scale_IKI-SD,Rt_scale_IRI-mean,Rt_scale_IRI-SD,Rt_long_scale_IKI-mean,Rt_long_scale_IKI-SD,Rt_long_scale_IRI-mean,Rt_long_scale_IRI-SD,Rt_arpeggio_IKI-mean,...,Lt_long_scale_IKI-mean,Lt_long_scale_IKI-SD,Lt_long_scale_IRI-mean,Lt_long_scale_IRI-SD,Lt_arpeggio_IKI-mean,Lt_arpeggio_IKI-SD,Lt_arpeggio_IRI-mean,Lt_arpeggio_IRI-SD,handedness,affected side
0,1,82.312112,13.229921,82.135133,28.898811,86.265478,13.311980,86.875652,39.062791,90.066294,...,95.666406,13.465412,96.125029,39.938112,101.131731,16.358171,102.647460,40.739545,R,NaN
1,1,88.946394,12.973927,89.013980,31.402664,98.032472,12.947017,105.671697,54.441392,118.003340,...,104.111900,14.654740,111.136122,58.183404,168.127512,16.548132,168.258340,27.776093,R,NaN
2,1,69.061144,13.557577,71.281947,28.322600,90.262109,11.066986,90.212710,25.212559,83.783431,...,96.978944,11.653787,96.649603,29.018465,97.928150,21.112888,98.374447,47.075235,R,NaN
3,1,104.627531,14.773290,103.568700,30.408687,128.489531,16.464423,127.964277,35.699352,150.809331,...,161.587047,17.913393,161.105616,40.423535,180.687931,16.687300,180.634780,35.095249,R,NaN
4,1,88.024163,15.166748,89.525860,22.041914,97.867809,12.792190,97.921832,30.024507,214.103394,...,114.973956,12.947909,114.820481,37.861518,224.061081,12.329798,224.257727,18.891626,R,NaN


In [12]:
# 全ての演奏関連特徴量の多重共線性チェック
performance_features = [
    # 右手の特徴量（平均値のみ）
    'Rt_scale_IKI-mean', 'Rt_long_scale_IKI-mean', 'Rt_arpeggio_IKI-mean',
    'Rt_scale_IRI-mean', 'Rt_long_scale_IRI-mean', 'Rt_arpeggio_IRI-mean',
    # 左手の特徴量（平均値のみ）
    'Lt_scale_IKI-mean', 'Lt_long_scale_IKI-mean', 'Lt_arpeggio_IKI-mean',
    'Lt_scale_IRI-mean', 'Lt_long_scale_IRI-mean', 'Lt_arpeggio_IRI-mean'
]

# VIF計算
X = df[performance_features]
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# VIFの結果を確認（値が大きい順にソート）
print(vif_data.sort_values('VIF', ascending=False))

                   Feature           VIF
11    Lt_arpeggio_IRI-mean  34638.007601
8     Lt_arpeggio_IKI-mean  33561.443742
5     Rt_arpeggio_IRI-mean  20331.275431
2     Rt_arpeggio_IKI-mean  19751.696341
4   Rt_long_scale_IRI-mean  10626.136611
1   Rt_long_scale_IKI-mean  10406.879366
7   Lt_long_scale_IKI-mean   8909.655697
10  Lt_long_scale_IRI-mean   8864.828207
0        Rt_scale_IKI-mean   8260.561555
3        Rt_scale_IRI-mean   8209.686043
9        Lt_scale_IRI-mean   6268.745018
6        Lt_scale_IKI-mean   6249.002407
